**Sets up environment**

In [5]:

import os
import time
import math
import pandas as pd
import requests
import re
from dotenv import load_dotenv
from iexfinance.stocks import Stock
from concurrent.futures import ThreadPoolExecutor as PoolExecutor

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

load_dotenv()

os.environ['IEX_TOKEN'] = os.environ.get('IEX_TOKEN')

with open('cookie.txt', 'r') as file:
  os.environ['COOKIE'] = file.read()


**Gets list of S&P 500 from Wikipedia and caches it to localhost**

In [6]:

try:
  tickers = []

  with open('s&p.txt', 'r') as filehandle:
    for ticker in filehandle:
      tickers.append(ticker[:-1])

except:
  html = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies').content
  stock_table = pd.read_html(html)
  stock_df = table[0]
  tickers = stock_df['Symbol'].to_list()

  with open('s&p.txt', 'w') as filehandle:
    for ticker in tickers:
        filehandle.write('%s\n' % ticker)

tickers


['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'T',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'CXO',

**Should only be ran the first time the program is ran to log into Investopedia**


In [0]:

import time

chrome_options = Options()
chrome_options.add_argument("--user-data-dir=chrome-data")

driver = webdriver.Chrome(options=chrome_options)
chrome_options.add_argument("user-data-dir=chrome-data") 

driver.get("https://investopedia.com/simulator/game/")
time.sleep(20)  # Time to enter credentials
driver.quit()


**Gets delayed Investopedia price**

In [0]:

def get_investopedia_price(symbol):
  headers = { 'cookie': os.environ['COOKIE'] }
  url = "https://www.investopedia.com/simulator/ajax/quotebox.aspx"
  body = { 'symbol': symbol }
  content = requests.post(url, body, headers=headers).content

  try:
    price = re.search("Last[\S\s]*\">(.*)<\/[\S\s]*>Change", content.decode()).group(1)
    return float(price.replace(',', ''))
  except:
    print(content)
    return -1


**Gets current price**

In [7]:

def get_current_price(symbol):
  return Stock(symbol).get_quote()['latestPrice']
  

**Shows symbols' price differences, descending**

In [0]:

def get_price_comparisons(syms):
  symbols = [syms] if isinstance(syms, str) else syms

  def get_price_comparison(sym):
    cur = get_current_price(sym)
    inv = get_investopedia_price(sym)
    pro = round(abs(cur - inv) / inv * 100, 2)
    ord = 'Buy' if cur > inv else 'Sell Short' if cur < inv else 'Even'
    amt = math.floor(50000 / cur)

    return [sym, inv, cur, pro, ord, amt]


  results = []
  with PoolExecutor(max_workers=5) as executor:
      for res in executor.map(get_price_comparison, symbols):
        results.append(res)


  df = pd.DataFrame(results, columns=['Symbol', 'Investopedia', 'Current', 'Profit %', 'Order', 'Amount'])
  sorted = df.sort_values('Profit %', ascending=False)

  return sorted


**Trades stocks**

In [0]:

def trade_stock(symbol, amount, order):
  chrome_options = Options()
  chrome_options.add_argument("--user-data-dir=chrome-data")
  chrome_options.add_argument('--headless')

  driver = webdriver.Chrome(options=chrome_options)
  driver.get('https://www.investopedia.com/simulator/trade/tradestock.aspx')

  def typeElement(selector, keys):
    driver.find_element_by_css_selector(selector).send_keys(keys)

  def setDropdownElement(id, value):
    driver.find_element_by_xpath('//select[@id="%s"]/option[text()="%s"]' % (id, value)).click()
  
  def clickElement(selector):
    elem = driver.find_element_by_css_selector(selector)
    driver.execute_script('arguments[0].click()', elem)

  typeElement('input[name="symbolTextbox"]', symbol)
  setDropdownElement("transactionTypeDropDown", order)
  typeElement('input[name="quantityTextbox"]', amount)

  clickElement('#sendConfirmationEmailCheckBox')
  clickElement('#previewButton')

  time.sleep(0.3)

  clickElement('#submitOrder')

  time.sleep(0.3)

  assert "Confirmation" in driver.page_source

  driver.quit()




---



In [13]:

get_investopedia_price('iep')


48.45

In [23]:
get_price_comparisons(["KNSA", "DLPH", "AKRO", "PRQR", "MX", "GEL", "CNQ", "NEWT", "PE", "SU", "NRZ", "AFIN", "DCP", "ERIE", "ETNB", "GNW"])

,Symbol,Investopedia,Current,Profit %,Order,Amount
12,DCP,3.7350,3.830,2.54,Buy,13054
15,GNW,3.5827,3.510,2.03,Sell Short,14245
2,AKRO,21.1300,20.850,1.33,Sell Short,2398
4,MX,11.4700,11.355,1.00,Sell Short,4403
8,PE,5.5050,5.560,1.00,Buy,8992
1,DLPH,8.0500,7.975,0.93,Sell Short,6269
5,GEL,3.5600,3.530,0.84,Sell Short,14164
11,AFIN,5.9200,5.870,0.84,Sell Short,8517
14,ETNB,25.0000,25.205,0.82,Buy,1983
6,CNQ,13.2400,13.135,0.79,Sell Short,3806


In [0]:
trade_stock("DCP", 13054, "Buy")